In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("apple_quality.csv", nrows=400)
display(df.head(2), df.sample(2), df.tail(2))

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.32984,-0.491590,good
1,1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.86753,-0.722809,good


,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
224,224,0.624839,-0.786826,0.907733,-0.350257,0.653573,-1.480451,2.569365,good
179,179,0.443372,-2.637919,0.319253,2.622455,2.551202,1.188443,-1.013236,good


,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
398,398,0.780136,-1.562171,0.867887,3.279170,3.136770,0.853514,-0.117891,good
399,399,-1.037014,2.611287,-2.975672,0.409859,-0.583967,3.587747,4.405949,bad


In [3]:
df = df.drop(columns={"A_id"})
df.head()

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,good
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,good
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,bad
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,good
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,good


In [4]:
# Dataset Information
display(df.describe(),
pd.DataFrame({
    'feature': df.columns.values,
    'dtypes': [df[col].dtype for col in df.columns],
    'n_unique': df.nunique().values,
    'n_nan': [df[col].isna().sum() for col in df.columns],
    'n_dupe': [df.duplicated().sum() for row in df.columns],
    'sample_unique': [df[col].unique() for col in df.columns]
    })
)
print(f'''
      Total Columns that having missing value = {df.isnull().any().sum()}
      Total Columns that are clean            = {df.shape[1] - df.isnull().any().sum()}
      Total Rows that having missing value    = {df.isnull().sum().sum()}
      Total Rows                              = {df.shape[0]}
      Totol Percentage of missing value      = {(df.isnull().sum().sum()/np.product(df.shape)) * 100}
      ''')

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,-0.564365,-1.004735,-0.430205,0.972296,0.650043,0.551777,0.292374
std,2.048663,1.534073,1.962900,1.523668,1.955560,1.993176,2.195753
min,-6.129668,-6.235107,-5.029716,-3.292343,-4.786379,-5.864599,-6.460989
25%,-2.110557,-2.030827,-1.733934,-0.111653,-0.689066,-0.672451,-1.403337
50%,-0.590392,-1.106041,-0.560745,0.945629,0.690460,0.631382,0.248145
75%,0.951110,-0.125534,0.883914,1.942342,2.034339,1.942203,1.923175
max,5.189624,5.149000,5.346330,7.619852,5.148739,6.346445,7.404736


,feature,dtypes,n_unique,n_nan,n_dupe,sample_unique
0,Size,float64,400,0,0,"[-3.970048523, -1.195217191, -0.292023862, -0...."
1,Weight,float64,400,0,0,"[-2.512336381, -2.839256528, -1.351281995, -2...."
2,Sweetness,float64,400,0,0,"[5.346329613, 3.664058758, -1.738429162, 1.324..."
3,Crunchiness,float64,400,0,0,"[-1.012008712, 1.588232309, -0.342615928, -0.0..."
4,Juiciness,float64,400,0,0,"[1.844900361, 0.853285795, 2.838635512, 3.6379..."
5,Ripeness,float64,400,0,0,"[0.329839797, 0.867530082, -0.038033328, -3.41..."
6,Acidity,float64,400,0,0,"[-0.491590483, -0.722809367, 2.621636473, 0.79..."
7,Quality,object,2,0,0,"[good, bad]"



      Total Columns that having missing value = 0
      Total Columns that are clean            = 8
      Total Rows that having missing value    = 0
      Total Rows                              = 400
      Totol Percentage of missing value      = 0.0
      


In [5]:
# ReType
df["Acidity"] = df['Acidity'].astype(float)

In [6]:
# Outliers
numeric_columns = df.select_dtypes(include=np.number).columns.tolist()
def find_anomalies(data, column_name):
    q1, q3 = data.quantile([0.25, 0.75])
    iqr = q3 - q1
    limit = iqr * 1.5
    bot = q1 - limit
    top = q3 + limit
    outliers_count = ((data < (bot)) | (data > (top))).sum()

    result = pd.DataFrame({
        'Column': [column_name],
        'IQR': [iqr],
        'Lower Bound': [bot],
        'Upper Bound': [top],
        'Outliers': [outliers_count]
    })
    return result

df_outliers = pd.DataFrame(columns=['Column', 'IQR', 'Lower Bound', 'Upper Bound', 'Outliers'])

for column in numeric_columns:
    result = find_anomalies(df[column], column)
    df_outliers = pd.concat([df_outliers, result], ignore_index=True)
    
# Remove Outliers
for i,low,up in zip(df_outliers.Column.unique(),df_outliers["Lower Bound"].unique(),df_outliers["Upper Bound"].unique()):
    df = df[(df[i] >= low) & (df[i] <= up)]

display(df_outliers)
print(f"Numbers of Outliers Removed : {df_outliers['Outliers'].sum()}")

C:\Users\Utente\AppData\Local\Temp\ipykernel_12240\2631605100.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_outliers = pd.concat([df_outliers, result], ignore_index=True)


,Column,IQR,Lower Bound,Upper Bound,Outliers
0,Size,3.061667,-6.703057,5.543610,0
1,Weight,1.905293,-4.888767,2.732406,7
2,Sweetness,2.617848,-5.660705,4.810686,3
3,Crunchiness,2.053995,-3.192646,5.023335,4
4,Juiciness,2.723405,-4.774174,6.119447,1
5,Ripeness,2.614654,-4.594431,5.864183,7
6,Acidity,3.326511,-6.393104,6.912942,2


Numbers of Outliers Removed : 24


In [7]:
# Encode
df['Quality'] = df['Quality'].map({'bad': -1, 'good': 1})
df.sample(5)

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
306,-2.254441,-0.181272,-0.374981,-0.641099,3.268848,-1.137355,3.739759,-1
240,-2.279312,-2.127286,-1.056731,1.487951,1.480484,2.653676,2.662126,-1
142,1.259488,-0.557898,-2.578773,-0.488970,2.581219,-0.232605,0.489467,-1
106,-0.725181,-1.447134,1.840994,3.086182,2.229222,0.555091,3.040762,1
369,1.244043,-2.465735,-3.887530,2.233804,-0.618175,0.838173,-1.570154,1


### Single feature model

In [18]:
# Train Test Split
from sklearn.model_selection import train_test_split
#from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
X = df.drop("Quality", axis=1)
#X = pd.DataFrame(X[["Size", "Weight"]])
y = df["Quality"]

In [19]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
#smote = SMOTE(random_state=42)
#X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

#X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
print(f"""
X_train = {X_train.shape}
X_test = {X_test.shape}""")


X_train = (302, 7)
X_test = (76, 7)


In [20]:
# Import model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7763157894736842


In [21]:
w_star = np.insert(lr.coef_, 0, np.array(lr.intercept_))
print(w_star)

[-3.97234922  4.26360974  0.20499547  2.59368885  0.54940743  1.83380268
 -0.23547074 -1.35549975]


In [ ]:
X_train["Cost"] = np.ones(X_train.shape[0])
X_train.T

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(X_train["Size"].values, y_train.values)
plt.scatter(X_train["Size"].values, 1 / (1 + np.exp(- np.dot(w_star.T, X_train.values.T))))
plt.show()

In [ ]:
# Generate a neighbour point
eps = 0.1
w_bar = w_star + np.array([0.5,-0.5,0,0,0,-0.08,0.5,-0.7])
print(np.linalg.norm(w_star - w_bar))
print(w_bar)

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, y_pred)